<a href="https://colab.research.google.com/github/julianVelandia/FinetuningLLMGradeWorksUNALDatasetInstruct/blob/master/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/user-attachments/assets/a200c62f-b3d0-4c44-a956-f19f268fc543" alt="image" width="400"/>


In [1]:
!pip install datasets
!pip install peft transformers torch
!pip install -U bitsandbytes accelerate

REINICIAR EL COLAB POR ESTE ERROR: https://discuss.huggingface.co/t/importerror-in-bitsandbytes-with-accelerate/80060

In [3]:
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
DATASET_NAME = "JulianVelandia/unal-repository-dataset-train-instruct"
MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
OUTPUT_DIR = "/content/drive/MyDrive/Llama-3-8B-unal-instruct-ft"

LORA_RANK = 2
LORA_ALPHA = 8
LORA_DROPOUT = 0.2
HUGGINGFACE_TOKEN = ":)"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [5]:
from huggingface_hub import login

login(token=HUGGINGFACE_TOKEN)

dataset = load_dataset(DATASET_NAME, split="train")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/875 [00:00<?, ?B/s]

finetuning_dataset.json:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12525 [00:00<?, ? examples/s]

In [6]:
for idx, row in enumerate(dataset):
    print(f"Fila {idx + 1}: Tipo - {type(row)}, Contenido - {repr(row)}")
    if idx == 4:
        break


dataset_info = {
    "Tipo": type(dataset),
    "Tamaño": len(dataset),
    "Características": dataset.column_names,
}
print(dataset_info)

Fila 1: Tipo - <class 'dict'>, Contenido - {'prompt': '¿Qué tipo de diseño de estudio se está utilizando para investigar la enfermedad nosocomial NAVM en el hospital?', 'completion': 'Diseño anidado en cohorte.', 'fragment': 'con diagnostico confirmado de NAVM. Este comité multidisciplinario, se encuentra funcionando desde el 15 de abril del 2016 y usa criterios internacionalmente aceptados para el diagnóstico de NAVM, además la NAVM como enfermedad nosocomial hace parte de los reportes oficiales de vigilancia epidemiológica que el hospital debe informar al Instituto Nacional de Salud (INS) y que se llenan de forma rutinaria en fichas epidemiológicas según lo establecido por el Ministerio de la Protección social en Colombia desde el año 2016. Todos los pacientes con diagnóstico de NAVM serán incluidos como casos verificando la presencia de criterios de inclusión y exclusión, los controles se tomarán de forma aleatoria (4 por cada caso) al momento de la aparición temporal del caso (dise

In [7]:
!nvidia-smi
!nvcc --version


Fri Feb  7 02:54:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
from bitsandbytes import __version__
print("Versión de bitsandbytes instalada:", __version__)

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
token = HUGGINGFACE_TOKEN

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

Versión de bitsandbytes instalada: 0.45.2


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
def preprocess_function(examples):
    inputs = tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples["completion"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["fragment"])

print("\nDataset tokenizado:")
print(tokenized_dataset)

Map:   0%|          | 0/12525 [00:00<?, ? examples/s]


Dataset tokenizado:
Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12525
})


In [10]:
from transformers import TrainerCallback

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=250,
    save_total_limit=2,
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_dir=f"{OUTPUT_DIR}/logs",
    fp16=True,
    push_to_hub=False
)
last_checkpoint = None
if os.path.exists(OUTPUT_DIR):
    checkpoints = [d for d in os.listdir(OUTPUT_DIR) if d.startswith("checkpoint")]
    if checkpoints:
        last_checkpoint = os.path.join(OUTPUT_DIR, max(checkpoints, key=lambda x: int(x.split('-')[-1])))
if last_checkpoint:
    print(f"Resumiendo desde el último checkpoint: {last_checkpoint}")
else:
    print("No se encontró un checkpoint previo. Iniciando desde cero.")


class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        print(f"Checkpoint guardado en {args.output_dir} en el paso {state.global_step}")
        checkpoints = [d for d in os.listdir(args.output_dir) if d.startswith("checkpoint")]
        print(f"Checkpoints actuales: {checkpoints}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    callbacks=[SaveCheckpointCallback()]
)



Resumiendo desde el último checkpoint: /content/drive/MyDrive/Llama-3-8B-unal-instruct-ft/checkpoint-4500


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-9edbd6f0b3d1>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
print("Entrenando")
if last_checkpoint:
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

Entrenando


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jvelandiag (julian-unal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.load(rng_file)


Step,Training Loss


Checkpoint guardado en /content/drive/MyDrive/Llama-3-8B-unal-instruct-ft en el paso 4695
Checkpoints actuales: ['checkpoint-4500', 'checkpoint-4695']


In [14]:
print('subir')
login(token=HUGGINGFACE_TOKEN)
repo_id = "JulianVelandia/Llama-3-8B-unal-instruct-ft"
model.save_pretrained(OUTPUT_DIR, push_to_hub=True, repo_id=repo_id)
tokenizer.save_pretrained(OUTPUT_DIR, push_to_hub=True, repo_id=repo_id)

subir


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

('/content/drive/MyDrive/Llama-3-8B-unal-instruct-ft/tokenizer_config.json',
 '/content/drive/MyDrive/Llama-3-8B-unal-instruct-ft/special_tokens_map.json',
 '/content/drive/MyDrive/Llama-3-8B-unal-instruct-ft/tokenizer.json')